プログラムの流れ
1. 2000年からの履歴を取得
1. 最新のFX情報を取得して、Dataframeに保存
1. データフレームの結合
1. ファイルの保存

In [ ]:
#!pip3 install yfinance

In [8]:

import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import os

class Get_data():
    start = datetime(2000, 1, 1)
    end = datetime.now()

    def nikkei_download(self):
        # Use yfinance to download Nikkei 225 data
        df_nikkei = yf.download('^N225', start=self.start, end=self.end)
        print("配列長さ:", len(df_nikkei))
        df_nikkei = df_nikkei[['Open', 'Close']]
        df_nikkei = df_nikkei.reset_index()
        return df_nikkei

    def dow_dayplus(self, row):
        # Explicitly cast weekday to int
        wkday = int(row['weekday'])
        current_date = row['Date']
        
        # Use membership testing with a scalar integer
        if wkday in [0, 1, 2, 3]:
            current_date = current_date + timedelta(days=1)
        elif wkday == 4:
            current_date = current_date + timedelta(days=3)
            
        return current_date

    def dow_download(self):
        # Use yfinance to download Dow Jones Industrial Average data
        df_dow = yf.download('^DJI', start=self.start, end=self.end)
        df_dow['dow_compare'] = df_dow['Close'] / df_dow['Open']
        df_dow = df_dow[['dow_compare']]
        df_dow = df_dow.reset_index()

        df_dow['weekday'] = df_dow['Date'].apply(lambda x: x.weekday())
        df_dow['Date'] = df_dow.apply(self.dow_dayplus, axis=1)
        return df_dow

    def add_today_open(self, df_nikkei, open_value):
        last_day = pd.to_datetime(df_nikkei.tail(1)['Date'].values[0])
        one_weekday_after = last_day + relativedelta(weekday=0)

        df_nikkei = df_nikkei.append(
            {'Date': one_weekday_after, 'Open': open_value, 'Close': 0},
            ignore_index=True
        )
        return df_nikkei

    def main(self, today=""):
        # 日経平均取得
        df_nikkei = self.nikkei_download()
        print("日経平均【完了】")

        if today != "":
            df_nikkei = self.add_today_open(df_nikkei, today)

        # ダウ平均取得
        df_dow = self.dow_download()
        print("ダウ平均【完了】")

        # 結合
        df = pd.merge(df_nikkei, df_dow, on='Date', how='inner')
        df['Close'] = df['Close'].astype('float')
        df['Open'] = df['Open'].astype('float')
        df = df[['Date', 'Open', 'Close', 'dow_compare']]
        print("結合【完了】")

        # ディレクトリが存在するかチェックして保存
        if not os.path.exists('data'):
            os.makedirs('data')
        df.to_pickle('data/stock.pkl')
        print("保存【完了】")
        display(df.tail(3))

        return df

g = Get_data()
g.main("")


[*********************100%***********************]  1 of 1 completed


配列長さ: 6131
日経平均【完了】


[*********************100%***********************]  1 of 1 completed
/var/folders/bh/r3p0802x4c3_5hc0jkwmdw640000gp/T/ipykernel_37492/857307656.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  wkday = int(row['weekday'])


ダウ平均【完了】
結合【完了】
保存【完了】


/var/folders/bh/r3p0802x4c3_5hc0jkwmdw640000gp/T/ipykernel_37492/857307656.py:67: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = pd.merge(df_nikkei, df_dow, on='Date', how='inner')


Price,Date,Open,Close,dow_compare
Ticker,,^N225,^N225,
5917,2025-01-07,39584.359375,40083.300781,0.996989
5918,2025-01-08,39879.359375,39981.058594,0.993428
5919,2025-01-09,39888.910156,39605.089844,1.002188


Price,Date,Open,Close,dow_compare
Ticker,,^N225,^N225,
0,2000-01-04,18937.449219,19002.859375,0.987451
1,2000-01-05,19003.509766,18542.550781,0.969002
2,2000-01-06,18574.009766,18168.269531,1.012128
3,2000-01-07,18194.050781,18193.410156,1.012588
4,2000-01-11,18246.099609,18850.919922,1.003444
...,...,...,...,...
5915,2024-12-30,40325.781250,39894.539062,0.996519
5916,2025-01-06,39945.421875,39307.050781,1.005562
5917,2025-01-07,39584.359375,40083.300781,0.996989
